## Decision Trees

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from utils import *

In [ ]:
plot_entropy()

### Dataset

|                                                     |   Ear Shape | Face Shape | Whiskers |   Cat  |
|:---------------------------------------------------:|:---------:|:-----------:|:---------:|:------:|
| <img src="images/0.png" alt="drawing" width="50"/> |   Pointy   |   Round     |  Present  |    1   |
| <img src="images/1.png" alt="drawing" width="50"/> |   Floppy   |  Not Round  |  Present  |    1   |
| <img src="images/2.png" alt="drawing" width="50"/> |   Floppy   |  Round      |  Absent   |    0   |
| <img src="images/3.png" alt="drawing" width="50"/> |   Pointy   |  Not Round  |  Present  |    0   |
| <img src="images/4.png" alt="drawing" width="50"/> |   Pointy   |   Round     |  Present  |    1   |
| <img src="images/5.png" alt="drawing" width="50"/> |   Pointy   |   Round     |  Absent   |    1   |
| <img src="images/6.png" alt="drawing" width="50"/> |   Floppy   |  Not Round  |  Absent   |    0   |
| <img src="images/7.png" alt="drawing" width="50"/> |   Pointy   |  Round      |  Absent   |    1   |
| <img src="images/8.png" alt="drawing" width="50"/> |    Floppy  |   Round     |  Absent   |    0   |
| <img src="images/9.png" alt="drawing" width="50"/> |   Floppy   |  Round      |  Absent   |    0   |


We will use **one-hot encoding** to encode the categorical features. They will be as follows:

- Ear Shape: Pointy = 1, Floppy = 0
- Face Shape: Round = 1, Not Round = 0
- Whiskers: Present = 1, Absent = 0

Therefore, we have two sets:

- `X_train`: for each example, contains 3 features:
            - Ear Shape (1 if pointy, 0 otherwise)
            - Face Shape (1 if round, 0 otherwise)
            - Whiskers (1 if present, 0 otherwise)
            
- `y_train`: whether the animal is a cat
            - 1 if the animal is a cat
            - 0 otherwise

In [ ]:
x_train=np.array([[1, 1, 1],
                  [0, 0, 1],
                  [0, 1, 0],
                  [1, 0, 1],
                  [1, 1, 1],
                  [1, 1, 0],
                  [0, 0, 0],
                  [1, 1, 0],
                  [0, 1, 0],
                  [0, 1, 0]])

y_train=np.array([1, 1, 0, 0, 1, 1, 0, 1, 0, 0])

In [ ]:
def calc_entropy(p):
  if p==0 or p==1:
    return 0
  else:
    return (-p*np.log2(p))-((1-p)*np.log2(1-p))

In [ ]:
print(calc_entropy(0.75))

In [ ]:
def split_indices(X, idx_feature):
  """Given a dataset and a index feature, return two lists for the two split nodes, the left node has the animals that have 
    that feature = 1 and the right node those that have the feature = 0 
    index feature = 0 => ear shape
    index feature = 1 => face shape
    index feature = 2 => whiskers
  """

  left_indices=[]
  right_indices=[]

  for i, x in enumerate(X):
    if x[idx_feature]==1:
      left_indices.append(i)
    else:
      right_indices.append(i)

  return left_indices, right_indices

### Computing based on Ear-Shape

In [ ]:
print(split_indices(x_train, 0))

In [ ]:
def weighted_entropy(X, y, left_idx, right_idx):
  w_left=len(left_idx)/len(X)
  w_right=len(right_idx)/len(X)

  p_left=sum(y[left_idx])/len(left_idx)
  p_right=sum(y[right_idx])/len(right_idx)

  w_entropy=w_left*calc_entropy(p_left)+w_right*calc_entropy(p_right)
  return w_entropy

In [ ]:
left_idx, right_idx=split_indices(x_train, 0)
print(weighted_entropy(x_train, y_train, left_idx, right_idx))

In [ ]:
def information_gain(X, y, left_idx, right_idx):
  p_root=sum(y)/len(y)
  h_node=calc_entropy(p_root)
  w_entropy=weighted_entropy(X, y, left_idx, right_idx)

  info_gain=h_node-w_entropy
  return info_gain

In [ ]:
print(information_gain(x_train, y_train, left_idx, right_idx))

In [ ]:
features=['Ear Shape', 'Face Shape', 'Whiskers']

for i, x in enumerate(features):
  left_idx, right_idx=split_indices(x_train, i)
  i_gain=information_gain(x_train, y_train, left_idx, right_idx)
  print(f"Feature: {x} ---> Information Gain if we split the root node using this feature: {i_gain}")

The feature with `highest` information gain must be used to split the root node.

In [ ]:
tree=[]
build_tree_recursive(x_train, y_train, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], "Root", max_depth=2, current_depth=0, tree=tree)

generate_tree_viz([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], y_train, tree)

The process is **recursive**, which means we must perform these calculations for each node until we meet a stopping criteria:

- If the tree depth after splitting exceeds a threshold
- If the resulting node has only 1 class
- If the information gain of splitting is below a threshold